In [ ]:
!pip install transformers sentencepiece sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AutoModel,
    AutoTokenizer
)
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
import numpy as np
import random
import sys
import os
import re

In [ ]:
def seed_everything(seed):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from numba import njit


@njit
def cosine_similarity(a, b) -> float:
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
def clean_text(text: str) -> str:
    bruh = {
        '(Г|г)(ор\.|ород|\.)': ' город ',
        '[Уу](л.|лица|ул)\s': ' улица ',
        '[Пп](роспект|р\.\s|р-кт\s|р\sкт\s|р\-т\s)': ' проспект ',
        '[Рр](айон|-н\s|-он\s)': ' район ',
        '[Пп](ос\.\s|оселок\s|осёлок\s)': ' поселок ',
        '[сC](танция\s|т\.\s|ст\s)': ' станция ',
        '[Уу](часток\s|ч\.\s|ч\s) ': ' участок ',
        '[Пп](ереулок\s|ер\.\s|ер\s)'
        '[Рр](ека\s|\.\s|\s)': ' река ',
        '[Пп](ер\.|ер|ереулок)\s': ' переулок ',
        '\s[Дд](ом|\.|)\s': ' дом ',
        '\s[Кк](орпус|\.|)\s': ' корпус ',
        '[Сс]тр(\.|оение|оен|оен\.)\s': ' строение ',
        '[Дд](ор\.|орога)\s': ' дорога ',
        '[Лл](итера|\.|ит\.)\s': ' литера ',
        '[Дд](еревня|ер\.)\s': ' деревня ',
        '[Бб](ульв\.|уль\.|ульвар)\s': ' бульвар ',
        '[Нн](аб\.|абереж\.|абережная)\s': ' набережная ',
        '[Сс](адовоство|сад\.|садовод\.)\s': ' садоводство ',
        'с(\/|\s*|\-|\.|)т(\.|)\s': ' Садоводческое товарищество ',
        '[Оо](стров|\.)\s': ' остров ',
        '[Кк](анал|ан\.)\s': ' канал',
        '[Жж](\/|\s*|\-|\.|)д(\.|)\s': ' железная дорога ',
        ' свх\.\s': ' совхоз ',
        ' потреб\. кооператив садоводства\s': ' потребительский кооператив садоводства ',
    }

    def substraction(a):
        a = re.sub('(Г|г|)(ор\.*|ород|\.|)\s*[Сс]анкт(-|\s*)[Пп]етербург', '', a)
        for i in bruh.keys():
            a = re.sub(i, bruh[i], a)
        a = a.lstrip().rstrip().lower()
        return re.sub(r'[-.?!)(,:\/\\]', ' ', a)

    return substraction(text)

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

In [ ]:
def make_labse(df, col, model=model):
  df_model = 'MiniLM'
  text_features = []
  for sentence in tqdm(df[col]):
    sentence_embeddings = list(model.encode([sentence]))
    text_features.extend(sentence_embeddings)
  text_features_df = pd.DataFrame(text_features, columns = [f'{df_model}_{col}_feature_{i}' for i in range(len(text_features[0]))])
  return text_features_df

def make_only_one_labse(address, model):
    sentence_embeddings = list(model.encode([address]))
    return sentence_embeddings[0]


In [ ]:
address = 'пр-кт Жукова, д. 30А'
address = clean_text(address)

same_address = 'жковаааааааааа 30А'
same_address = clean_text(same_address)

In [ ]:
%%time
embs = make_only_one_labse(address, model)
same_embs = make_only_one_labse(same_address, model)

CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 21.9 ms


In [ ]:
%%time
cosine_similarity(embs, same_embs)

CPU times: user 16 µs, sys: 3 µs, total: 19 µs
Wall time: 21.9 µs


0.7678707838058472

In [ ]:
import pandas as pd
df = pd.read_csv('/content/buildings.csv')

<ipython-input-20-20dc3d153904>:2: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/buildings.csv')


In [ ]:
df.head(2)

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,город пушкин кедринская улица дом 12,True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,поселок ушково пляжевая улица дом 4б,True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN


In [ ]:
filter_dataset = pd.DataFrame()

In [ ]:
filter_dataset['is_actual'] = df['is_actual']
filter_dataset['id'] = df['id']

In [ ]:
df['full_address'][149134] = df['short_address'][149134]

<ipython-input-24-6c1cd0cb62fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_address'][149134] = df['short_address'][149134]


In [ ]:
feature_table = make_labse(df, 'full_address')

  0%|          | 0/166645 [00:00<?, ?it/s]

In [ ]:
tqdm.pandas()
filter_dataset['embeddings'] = df['full_address'].progress_apply(make_labse)

  0%|          | 0/166645 [00:00<?, ?it/s]

In [ ]:
filter_dataset.to_csv('all_mlni_enmbeddings.csv')

In [ ]:
filter_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166645 entries, 0 to 166644
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   is_actual   166645 non-null  bool  
 1   id          166645 non-null  int64 
 2   embeddings  166645 non-null  object
dtypes: bool(1), int64(1), object(1)
memory usage: 2.7+ MB
